# Summary of Experiments

## gRPC service on AWS ECS with EC2 lunch type

The good news is that most of the recepie can be used from here:

https://dev.to/chaitan94/deploying-a-grpc-service-in-ecs-with-the-ec2-launch-type-2aa


I am just going to cover what was missing or different.


1- to push the image(s) into ECR I used this command:

```
aws ecr-public get-login-password --region us-east-1 | docker login --username AWS --password-stdin public.ecr.aws/p0c6a5h1
docker tag greeter_server:latest public.ecr.aws/p0c6a5h1/greeter_server:latest
docker push public.ecr.aws/p0c6a5h1/greeter_server:latest
```

first need to create a repository on ECR. The push commands can be get from there.


2- Step3: It seems that AWS has changed the requirments and settings of ECS clusters with EC2 lunch types.
The step three will not work as it is.
I changed the port mapping to 50051:50051
The networking type should be set as Brdige (which is the default option as well)

3- Step 4: I had to change the Target Group to HTTP with 50051 as port and gRPC as protcol version

4- To Create a Load Balancer (and arguablly this was the hardest part) with HTTPS listener, we need to buy/register a domain.
I got a domain from google domains, and next used ACM to issue a https certificate for it. More on this comes next.

5- for the load balancer in View/edit rules, for the listener I changed to "HTTPS : 50051". (I guess this is not a deal braker)
Also, It is really important that all of the resources that we define are in the same VPC and subnets.

6- In the cluster creation, cluster should be in the same VPC and have Same security group as ALB.

7- Step5 of tut;  this particular section is not valid any more:

> Create a new security group. (or select an existing one, if you already have one. Make sure 50051 port is allowed in the inbound rules.
Choose Edit next to Security groups.
For Inbound rules for security group, change the Type to Custom TCP and set the Port range to 50051. (Note: This is a bad security posture and only permissible for this demo. Your application should have more strict security rules.)

8- for the production listener port I chose:  "50051:HTTPS".

9- The target group as well was selected the same target group that was defined before.




## Setup of Domain on Google Domain and Connection to AWS ALB

1- after buying the domain in google domains, we need to go to ACM (in the ALB HTTPS listerner creation you are routed to ACM as well)

2- in the issue certificte section, chose the certificate with DNS option.

3- Important: provide the name of a sub-domain not the main domain. for example if the domain name is example.come,  provide a name like app.example.come.
This name does not have to be created or configured in the google side yet.

4- when ACM provides the CNAME record, it is in the form of _8ca938b1b9f418cc2eeqeq3e.app.example.com.
in the google domain, DNS section, we need to add a CNAME record. The record must be corrected in this form:
_8ca938b1b9f418cda6d81826.app (i.e. the example.com. pieace in the end should be removed)
the CNAME Value will not change.

5- it will take a few minutes to have this record propagated. can use this link to see the status:
https://www.whatsmydns.net/#NS/boarlabs.net

6- After ACM issued the certificate, you would proceed with ALB creation, and will assign the certificate in the process.

7- after ALB is created it will have a description section, in which the DNS records is given
> DNS xxxxxxxxxxx.us-east-2.elb.amazonaws.com 
(A Record)

8 - will need to copy this address and place it in the DNS section of the google domain.
an easy way is to create another CNAME record that 
has 
Name = app  and the Value = ( The A record coppied from AWS above)

## AWS gRPC service with Fargate lunch type:

This one is arguably much simpler, but seems to be more expensive option.
the best links to follow for this are:

https://aws.amazon.com/blogs/aws/new-application-load-balancer-support-for-end-to-end-http-2-and-grpc/

I followed the most of each exactly as it says. Only for ALB need to repeat the steps in above to connect it to the ALB.
potentially one could use the same ALB, just add different HTTPS listeners on different ports, or different IF/Then Rules.
